# Qiskit ExactReciprocal

This notebook demonstrates the Qiskit `ExactReciprocal` operation.

* [Qiskit ExactReciprocal documentation](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.ExactReciprocal)

The documentation states that the `ExactReciprocal` operation takes the state $|x\rangle|0\rangle$ to:
$$
\cos(\frac{1}{x})|x\rangle|0\rangle + \sin(\frac{1}{x})|x\rangle|1\rangle
$$

As well as the number of qubits for the input value, the operation takes two further parameters:

* `scaling`: Scaling factor $s$ of the reciprocal function, i.e. to compute $s/x$.
* `neg_vals` (Boolean): Whether $x$ might represent negative values. In this case the first qubit is the sign, with $|1\rangle$ for negative and $|0\rangle$ for positive. For the negative case it is assumed that the remaining string represents $1-x$. This is because $e^{-2\pi ix}=e^{2\pi i(1-x)}$ for $x\in[0,1]$.

Jack Morgan's HHL circuit uses `neg_vals=True`.

How does the `ExactReciprocal` operation work? Let's examine some examples.

In [23]:
import numpy as np
from IPython.display import display, Markdown
from math import isclose, log2
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import ExactReciprocal, StatePreparation
from qiskit.circuit.library.generalized_gates import UCRYGate
from qiskit.quantum_info import Statevector


def create_basis_statevector_input_circuit(input):
    x = np.array(input) / np.linalg.norm(input)
    num_state_qubits = int(log2(x.size))
    state_preparation = StatePreparation(Statevector(x))
    x_reg = QuantumRegister(num_state_qubits, 'x')
    circuit = QuantumCircuit(x_reg)
    circuit.append(state_preparation, x_reg)
    return circuit


def create_exact_reciprocal_circuit(input):
    x = np.array(input) / np.linalg.norm(input)
    num_state_qubits = int(log2(x.size))
    scaling = 2**-num_state_qubits
    state_preparation = StatePreparation(Statevector(x))
    x_reg = QuantumRegister(num_state_qubits, 'x')
    a_reg = QuantumRegister(1, 'a')
    circuit = QuantumCircuit(a_reg, x_reg)
    circuit.append(state_preparation, x_reg)
    exact_reciprocal = ExactReciprocal(num_state_qubits=num_state_qubits, scaling=scaling)
    circuit.append(exact_reciprocal, x_reg[::-1] + [a_reg[0]])
    return circuit


def latex_circuit_output(circuit):
    simulator = Aer.get_backend('statevector_simulator')
    aer_circuit = transpile(circuit, simulator)
    return simulator.run(aer_circuit).result().get_statevector().draw(output='latex').data.replace('|', r'\|')


def run_table(basis):
    markdowntable = f"""
| Input | Output |
|--------------|------------------|
"""
    for vec in basis:
        input = latex_circuit_output(create_basis_statevector_input_circuit(vec))
        output = latex_circuit_output(create_exact_reciprocal_circuit(vec))
        markdowntable += f"| {input} | {output} |\n"
    display(Markdown(markdowntable))


basis2 = [
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
]

basis3 = [
    [1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1]
]

basis4 = [
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
]

input = latex_circuit_output(create_basis_statevector_input_circuit([0, 1, 0, 0, 0, 0, 0, 0]))
output = latex_circuit_output(create_exact_reciprocal_circuit([0, 1, 0, 0, 0, 0, 0, 0]))

markdowntable = f"""
| Input | Output |
|--------------|------------------|
| {input} | {output} |
"""

# print(markdowntable)

# print(exact_reciprocal.unitary)
# # exact_reciprocal.draw(output='mpl')

# statevector = run_statevector(circuit)
# statevector.draw(output='latex')

In [24]:
run_table(basis2)



| Input | Output |
|--------------|------------------|
| $$ \|00\rangle$$ | $$ \|000\rangle$$ |
| $$ \|01\rangle$$ | $$\frac{\sqrt{3}}{2} \|010\rangle+\frac{1}{2} \|011\rangle$$ |
| $$ \|10\rangle$$ | $$ \|101\rangle$$ |
| $$ \|11\rangle$$ | $$\frac{2 \sqrt{2}}{3} \|110\rangle+\frac{1}{3} \|111\rangle$$ |


In [25]:
run_table(basis3)


| Input | Output |
|--------------|------------------|
| $$ \|000\rangle$$ | $$ \|0000\rangle$$ |
| $$ \|001\rangle$$ | $$\frac{\sqrt{15}}{4} \|0010\rangle+\frac{1}{4} \|0011\rangle$$ |
| $$ \|010\rangle$$ | $$\frac{\sqrt{3}}{2} \|0100\rangle+\frac{1}{2} \|0101\rangle$$ |
| $$ \|011\rangle$$ | $$0.9860132972 \|0110\rangle+\frac{1}{6} \|0111\rangle$$ |
| $$ \|100\rangle$$ | $$ \|1001\rangle$$ |
| $$ \|101\rangle$$ | $$0.9797958971 \|1010\rangle+\frac{1}{5} \|1011\rangle$$ |
| $$ \|110\rangle$$ | $$\frac{2 \sqrt{2}}{3} \|1100\rangle+\frac{1}{3} \|1101\rangle$$ |
| $$ \|111\rangle$$ | $$0.9897433186 \|1110\rangle+\frac{1}{7} \|1111\rangle$$ |


In [26]:
run_table(basis4)


| Input | Output |
|--------------|------------------|
| $$ \|0000\rangle$$ | $$ \|00000\rangle$$ |
| $$ \|0001\rangle$$ | $$0.9921567416 \|00010\rangle+\frac{1}{8} \|00011\rangle$$ |
| $$ \|0010\rangle$$ | $$\frac{\sqrt{15}}{4} \|00100\rangle+\frac{1}{4} \|00101\rangle$$ |
| $$ \|0011\rangle$$ | $$0.9965217286 \|00110\rangle+\frac{1}{12} \|00111\rangle$$ |
| $$ \|0100\rangle$$ | $$\frac{\sqrt{3}}{2} \|01000\rangle+\frac{1}{2} \|01001\rangle$$ |
| $$ \|0101\rangle$$ | $$0.9949874371 \|01010\rangle+\frac{1}{10} \|01011\rangle$$ |
| $$ \|0110\rangle$$ | $$0.9860132972 \|01100\rangle+\frac{1}{6} \|01101\rangle$$ |
| $$ \|0111\rangle$$ | $$0.9974457174 \|01110\rangle+\frac{1}{14} \|01111\rangle$$ |
| $$ \|1000\rangle$$ | $$ \|10001\rangle$$ |
| $$ \|1001\rangle$$ | $$0.99380799 \|10010\rangle+\frac{1}{9} \|10011\rangle$$ |
| $$ \|1010\rangle$$ | $$0.9797958971 \|10100\rangle+\frac{1}{5} \|10101\rangle$$ |
| $$ \|1011\rangle$$ | $$0.9970370305 \|10110\rangle+\frac{1}{13} \|10111\rangle$$ |
| $$ \|1100\rangle$$ | $$\frac{2 \sqrt{2}}{3} \|11000\rangle+\frac{1}{3} \|11001\rangle$$ |
| $$ \|1101\rangle$$ | $$0.9958591955 \|11010\rangle+\frac{1}{11} \|11011\rangle$$ |
| $$ \|1110\rangle$$ | $$0.9897433186 \|11100\rangle+\frac{1}{7} \|11101\rangle$$ |
| $$ \|1111\rangle$$ | $$0.9977753031 \|11110\rangle+\frac{1}{15} \|11111\rangle$$ |
